In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
from itertools import combinations
from math import factorial
import pandas as pd
import numpy as np
import sys, csv
import random
import time
import csv

In [5]:
class Graph(object):
    def __init__(self, nodes, init_graph):
        self.nodes = nodes
        self.graph = self.construct_graph(nodes, init_graph)
        
    def construct_graph(self, nodes, init_graph):
        '''
        Этот метод обеспечивает симметричность графика. Другими словами, если существует путь от узла A к B со значением V, должен быть путь от узла B к узлу A со значением V.
        '''
        graph = {}
        for node in nodes:
            graph[node] = {}
        
        graph.update(init_graph)
        
        for node, edges in graph.items():
            for adjacent_node, value in edges.items():
                if graph[adjacent_node].get(node, False) == False:
                    graph[adjacent_node][node] = value
        return graph
    
    def get_nodes(self):
        "Возвращает узлы графа"
        return self.nodes
    
    def get_outgoing_edges(self, node):
        "Возвращает соседей узла"
        connections = []
        for out_node in self.nodes:
            if self.graph[node].get(out_node, False) != False:
                connections.append(out_node)
        return connections
    
    def value(self, node1, node2):
        "Возвращает значение ребра между двумя узлами."
        return self.graph[node1][node2]
def dijkstra_algorithm(graph, start_node):
    unvisited_nodes = list(graph.get_nodes())
    # Мы будем использовать этот словарь, чтобы сэкономить на посещении каждого узла и обновлять его по мере продвижения по графику 
    shortest_path = {}
    
    # Мы будем использовать этот словарь, чтобы сохранить кратчайший известный путь к найденному узлу
    previous_nodes = {}
    
    k = 1 # номер итерации
    #print('\n')
    
    # Мы будем использовать max_value для инициализации значения "бесконечности" непосещенных узлов   
    max_value = sys.maxsize
    for node in unvisited_nodes:
        shortest_path[node] = max_value
    # Однако мы инициализируем значение начального узла 0  
    shortest_path[start_node] = 0
    
    # Алгоритм выполняется до тех пор, пока мы не посетим все узлы
    while unvisited_nodes:
        # Приведенный ниже блок кода находит узел с наименьшей оценкой
        current_min_node = None
        for node in unvisited_nodes: # Итерация по узлам
            if current_min_node == None:
                current_min_node = node
            elif shortest_path[node] < shortest_path[current_min_node]:
                current_min_node = node
            #print('shortest_path ',shortest_path)
        
        # Приведенный ниже блок кода извлекает соседей текущего узла и обновляет их расстояния
        neighbors = graph.get_outgoing_edges(current_min_node)
        #print('neighbors', neighbors, ' unvisited_nodes ', unvisited_nodes)
        for neighbor in neighbors:
            tentative_value = shortest_path[current_min_node] + graph.value(current_min_node, neighbor)
            if tentative_value < shortest_path[neighbor]:
                shortest_path[neighbor] = tentative_value
                # Мы также обновляем лучший путь к текущему узлу
                previous_nodes[neighbor] = current_min_node
            #print('tentative_value ', tentative_value, '\n')
            
            print('Итерация №{}\nНаименьший путь к точке {} равен {}'.format(k, neighbor, tentative_value))
            k += 1
            time.sleep(0.5)
            
        # После посещения его соседей мы отмечаем узел как "посещенный"
        unvisited_nodes.remove(current_min_node)
    return previous_nodes, shortest_path

def print_result(previous_nodes, shortest_path, start_node, target_node):
    path = []
    node = target_node
    
    while node != start_node:
        path.append(node)
        node = previous_nodes[node]
    
    # Добавить начальный узел вручную
    path.append(start_node)
    print("\nПолученный путь: {}. Стоимость: {}".format("-".join(reversed(path)), shortest_path[target_node]))


def output_nodes_init_graph(matrix):
    # заполняем список узлов значениями (имена узлов)
    nodes = []
    for i in range(len(matrix[0])):
        nodes.append(str(i+1))

    # Формирования словаря с ключами-узлами графа
    init_graph = {}
    for node in nodes:
        init_graph[node] = {}

    # Весовая матрица matrix
    matrix = pd.DataFrame(matrix)
    matrix.columns = nodes
    matrix.index = nodes
    print("Весовая матрица: \n", matrix)
    
    # Заполняем словарь init_graph значениями
    for start_value, row in enumerate(matrix, start=1):
        for value in range(start_value, len(matrix.columns)+1):
            if int(matrix[row][value-1]) == 0:
                continue
            else:
                init_graph[row][str(value)] = float(matrix[row][value-1])
    
    return nodes, init_graph


# ---------------------------------------------------------------------------


way = int(input('Каким способом вы хотите ввести данные? '))
# Ручной ввод
if way == 1:
    nodes = input('Введите узлы графа через запятую (без пробела): ')
    nodes = nodes.split(',')
    
    # Формирования словаря с ключами-узлами графа
    init_graph = {}
    for node in nodes:
        init_graph[node] = {}
        
    # Число сочетаний
    max_sum_edge = factorial(len(nodes)) / (2 * factorial(len(nodes) - 2))
    sum_edge = int(input('Количество ребер графа (не более {}): '.format(int(max_sum_edge))))
    
    # Ввод весовых коэффициентов и помещение их в словарь nodes
    for i in range(sum_edge):
        edge = input('Введите весовые коэф. в формате: A-B-n, где А - начальная точка, В - конечная, n - цена: ')
        edge = edge.split('-')
        init_graph[edge[0]][edge[1]] = int(edge[2])


elif way == 2:
    sum_rows = int(input('Введите количество узлов графа: '))
    
    matrix = []
    for i in range(sum_rows):
        input_rows = input('Введите {} сроку весовой матрицы (значения через запятую): '.format(i+1))
        input_rows = input_rows.split(',')
        matrix.append(input_rows)
        
    nodes, init_graph = output_nodes_init_graph(matrix)
    
# Случайная генерация
elif way == 3:
    sum_nodes = int(input('Введите количество узлов графа: '))
    
    # заполняем список узлов значениями (имена узлов)
    nodes = []
    for i in range(sum_nodes):
        nodes.append(str(i+1))
    
    max_sum_edge = (factorial(sum_nodes)) / (2 * factorial(sum_nodes - 2)) # Число сочетаний узлов графа
    sum_edge = int(input('Введите количество ребер графа (не более {}): '.format(int(max_sum_edge))))
    comb = list(combinations(nodes, 2)) # Сочетания узлов графа
    
    # Формирования словаря с ключами-узлами графа
    init_graph = {}
    for node in nodes:
        init_graph[node] = {}
    
    # Заполняем словарь случайными весовыми коэффициентами
    matrix = np.zeros((sum_nodes,sum_nodes)) # весовая матрица
    for i in range(sum_edge):
        random_edge = random.choice(comb)
        edge_rate = random.randint(0,100)
        matrix[int(random_edge[0]) - 1][int(random_edge[1]) - 1] = edge_rate
        matrix[int(random_edge[1]) - 1][int(random_edge[0]) - 1] = edge_rate
        
        init_graph[random_edge[0]][random_edge[1]] = edge_rate
        comb.remove(random_edge)
    
    matrix = pd.DataFrame(matrix)
    matrix.columns = nodes
    matrix.index = nodes
    print('Весовая матрица: \n', matrix)
    

elif way == 4:
    matrix = []
    file_way = input('Введите путь к файлу: ')
    with open(file_way, 'r', newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=';')
        for row in spamreader:
            matrix.append(row)
    nodes, init_graph = output_nodes_init_graph(matrix)

start_node_input = input('Начальная точка: ')
target_node_input = input('Конечная точка: ')


graph = Graph(nodes, init_graph)

previous_nodes, shortest_path = dijkstra_algorithm(graph=graph, start_node=start_node_input)
print_result(previous_nodes, shortest_path, start_node=start_node_input, target_node=target_node_input)

Каким способом вы хотите ввести данные? 1
Введите узлы графа через запятую (без пробела): А,1,2,3,4,5,6,7,8,9,В
Количество ребер графа (не более 55): 22
Введите весовые коэф. в формате: A-B-n, где А - начальная точка, В - конечная, n - цена: А-1-5
Введите весовые коэф. в формате: A-B-n, где А - начальная точка, В - конечная, n - цена: А-2-1
Введите весовые коэф. в формате: A-B-n, где А - начальная точка, В - конечная, n - цена: А-4-5
Введите весовые коэф. в формате: A-B-n, где А - начальная точка, В - конечная, n - цена: 1-3-3
Введите весовые коэф. в формате: A-B-n, где А - начальная точка, В - конечная, n - цена: 2-5-13
Введите весовые коэф. в формате: A-B-n, где А - начальная точка, В - конечная, n - цена: 2-4-3
Введите весовые коэф. в формате: A-B-n, где А - начальная точка, В - конечная, n - цена: 4-2-3
Введите весовые коэф. в формате: A-B-n, где А - начальная точка, В - конечная, n - цена: 4-7-6
Введите весовые коэф. в формате: A-B-n, где А - начальная точка, В - конечная, n - цен